<a href="https://colab.research.google.com/github/jerryHeisenberg/Final-Projects/blob/main/Autism_in_Children.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install comet_ml > /dev/null 2>&1

In [32]:
from comet_ml import Experiment
import tensorflow as tf
import numpy as np
import os
import cv2
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [48]:
def load_images_data_from_directory(directory, target_size=(256,256)):
  images=[]
  labels =[]
  for root,_, files in os.walk(directory):
    for filename in files:
      if filename.endswith(".jpg"):
        label_name = os.path.basename(os.path.dirname(root))
        label= 1 if label_name == "Autistic" else 0
        img_path = os.path.join(root,filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, target_size)
        img =img.astype('float32')/255.0
        images.append(img)
        labels.append(label)
  return np.array(images), np.array(labels)


In [6]:
experiment = Experiment(api_key = "fjasnw5RFV2HeIVIdLDhHpdyk",
                        project_name="Autism Detection System")


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jerryheisenberg/autism-detection-system/c44bed3ba8a646d2843bb59cd7b3d141



In [39]:
experiment.log_parameter("training_data_path", "/content/drive/MyDrive/AutismDataset/train")
experiment.log_parameter("validation_data_path","/content/drive/MyDrive/AutismDataset/valid")
experiment.log_parameter("test_data_path","/content/drive/MyDrive/AutismDataset/test")


In [50]:
train_images,train_labels = load_images_data_from_directory("/content/drive/MyDrive/AutismDataset/train")
print("train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
valid_images,valid_labels = load_images_data_from_directory("/content/drive/MyDrive/AutismDataset/valid")
print("Valid images shape:", valid_images.shape)
print("Valid labels shape:", valid_labels.shape)
test_images,test_labels = load_images_data_from_directory("/content/drive/MyDrive/AutismDataset/test")
print("test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

train images shape: (2540, 256, 256, 3)
Train labels shape: (2540,)
Valid images shape: (100, 256, 256, 3)
Valid labels shape: (100,)
test images shape: (300, 256, 256, 3)
Test labels shape: (300,)


In [45]:
print("test data:", os.listdir("/content/drive/MyDrive/AutismDataset/train"))

test data: ['Autistic.994.jpg', 'Non_Autistic.180.jpg', 'Autistic.987.jpg', 'Non_Autistic.290.jpg', 'Non_Autistic.1160.jpg', 'Non_Autistic.2.jpg', 'Non_Autistic.1116.jpg', 'Non_Autistic.1056.jpg', 'Autistic.990.jpg', 'Non_Autistic.1027.jpg', 'Non_Autistic.218.jpg', 'Non_Autistic.266.jpg', 'Non_Autistic.224.jpg', 'Non_Autistic.1243.jpg', 'Non_Autistic.193.jpg', 'Non_Autistic.1179.jpg', 'Non_Autistic.1137.jpg', 'Non_Autistic.1221.jpg', 'Non_Autistic.1076.jpg', 'Non_Autistic.1046.jpg', 'Non_Autistic.1058.jpg', 'Non_Autistic.1201.jpg', 'Autistic.92.jpg', 'Non_Autistic.1149.jpg', 'Non_Autistic.191.jpg', 'Non_Autistic.1086.jpg', 'Non_Autistic.229.jpg', 'Non_Autistic.1226.jpg', 'Non_Autistic.1072.jpg', 'Non_Autistic.1125.jpg', 'Non_Autistic.230.jpg', 'Non_Autistic.271.jpg', 'Autistic.986.jpg', 'Non_Autistic.1153.jpg', 'Non_Autistic.115.jpg', 'Non_Autistic.1184.jpg', 'Autistic.99.jpg', 'Non_Autistic.21.jpg', 'Non_Autistic.302.jpg', 'Non_Autistic.1235.jpg', 'Autistic.926.jpg', 'Autistic.920.jpg

In [51]:
#U-net
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation ='relu', input_shape=(256,256,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [52]:
#compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
#log
model.summary(print_fn=lambda x:experiment.log_text(x))


In [54]:
history = model.fit(
    train_images,train_labels,
    epochs=10,
    batch_size= 32,
    validation_data=(valid_images, valid_labels),
    callbacks=[experiment.get_callback('keras')]
)



COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/10
80/80 [==============================] - 425s 5s/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
80/80 [==============================] - 421s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
80/80 [==============================] - 412s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
80/80 [==============================] - 418s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
80/80 [==============================] - 410s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
80/80 [==============================] - 416s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
80/80 [==============================] - 414s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_l

In [55]:
test_loss, test_acc = model.evaluate(test_images,test_labels)
print(f'Test accuracy: {test_acc}')


10/10 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000
Test accuracy: 1.0


In [56]:
experiment.log_metric("test_accuracy", test_acc)

In [57]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/jerryheisenberg/autism-detection-system/c44bed3ba8a646d2843bb59cd7b3d141
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy                : 1.0
COMET INFO:     batch_accuracy          : 1.0
COMET INFO:     batch_loss [80]         : (0.0, 0.6655035018920898)
COMET INFO:     epoch_duration [10]     : (410.26544262400057, 424.8255151049998)
COMET INFO:     loss [10]               : (0.0, 0.008384299464523792)
COMET INFO:     test_accuracy           : 1.0
COMET INFO:     val_accuracy            : 1.0
COMET INFO:     val_loss                : 0.0
COMET INFO:     validate_batch_accuracy : 1.0
COMET INFO:     validate_